<img src="assets/pipelines_img.jpg" width=400 height=100 />

<center> <a href="https://unsplash.com/photos/Bu4lHKIHr-E">Source</a> </center>

# Pipeline for Data Validation and Model Building and Serving

Author: CHNG Soon Siang ([LinkedIn](https://www.linkedin.com/in/soon-siang-chng/))<br>
Dataset: [Heart Disease Dataset](https://www.kaggle.com/ronitf/heart-disease-uci)

Date last updated: 16 Oct 2021

_**Summary**_    

Machine Learning Operation (MLOps) is the marriage of DevOps with Machine Learning, to bring a machine learning model into production. In this notebook, with codes adapted from various sources, I explored how one can utilize a popular MLOps framework - Tensorflow Extended (TFX) - to do end-to-end machine learning on the Heart Disease dataset.

## Background



In [ ]:
import os
import pprint
import numpy as np
import urllib
import tempfile
import cloudpickle

import absl
import pandas as pd
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx

import tensorflow_data_validation as tfdv
import tensorflow_transform as tft

# TFX Components
from tfx.components import CsvExampleGen
from typing import Dict, List, Text
from tfx.components import Evaluator, ExampleValidator, Pusher, SchemaGen, StatisticsGen, Trainer, Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from google.protobuf.json_format import MessageToDict

# TFX Orchestrator and Proto
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

# Utilities
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from google.protobuf.json_format import MessageToDict
from tfx.types import standard_artifacts
import apache_beam as beam

# Display versions of TF and TFX related packages
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))
print('TensorFlow Data Validation version: {}'.format(tfdv.__version__))
print('TensorFlow Transform version: {}'.format(tft.__version__))
print(f'Apache Beam version: {beam.__version__}')

In [ ]:
!mkdir {os.path.join('.', 'HD_BNN_tfx')}

In [ ]:
## Define all constant

_tfx_root = os.path.abspath(os.path.join('.', 'HD_BNN_tfx'))     # Create location ~/tfx
_pipeline_root = os.path.join(_tfx_root, 'pipelines')   # Join ~/tfx/pipelines/
_metadata_db_root = os.path.join(_tfx_root, 'metadata.db')  # Join ~/tfx/metadata.db
_log_root = os.path.join(_tfx_root, 'logs')
_model_root = os.path.join(_tfx_root, 'model')
_data_root = os.path.join(_tfx_root, 'data')
_serving_model_dir = os.path.join(_tfx_root, 'serving_model')
_data_filepath = os.path.join(_data_root, "heart.csv")

# Modules for Transform component
_constant_transform_module_file = 'constants_transform.py'
_features_transform_module_file = 'features_transform.py'

# Modules for Trainer component
_input_fn_module_file = 'inputfn_trainer.py'
_constants_train_module_file = 'constants_trainer.py'
_model_trainer_module_file = 'model_trainer.py'

In [ ]:
context = InteractiveContext(pipeline_root=_tfx_root)

## ExampleGen

In [ ]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = CsvExampleGen(
    input_base = _data_root
)

# Run the component using the InteractiveContext instance
context.run(example_gen, enable_cache = False)

In [ ]:
len(example_gen.outputs['examples'].get()), type(example_gen.outputs['examples'].get())

In [ ]:
pp.pprint(example_gen.outputs['examples'].get()[0])

In [ ]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(artifact.uri, 'Split-train') # train -> Split-train

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# Utility function to convert TFRecords into dictionary for easy visualisation.

def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []

    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [ ]:
pp.pprint(get_records(dataset, 3))

## StatisticsGen

In [ ]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen( # Instantiate a StatisticsGen class
    examples=example_gen.outputs['examples'] # Insert the outputs of example_gen component into StatisticsGen
)
    
# Run the component
context.run(statistics_gen)

In [ ]:
# Plot the statistics generated
context.show(statistics_gen.outputs['statistics'])

## SchemaGen

In [ ]:
# Instantiate SchemaGen with the output statistics from the StatisticsGen
schema_gen = SchemaGen(
    statistics = statistics_gen.outputs['statistics'],
    # How to know the key is 'statistics'? Can view from the context.run output. >>>
    #.component.outputs	['statistics']	Channel of type 'ExampleStatistics' (1 artifact) at 0x7f1628fb09a0
)
    
# Run the component
context.run(schema_gen)

In [ ]:
schema_gen.outputs['schema']

In [ ]:
# Visualize the output
context.show(schema_gen.outputs['schema'])

What does 'presence' and 'valency' mean?

Presence indicates whether the feature must be present in 100% of examples (required) or not (optional).
Valency indicates the number of values required per training example. In the case of categorical features, single indicates that each training example must have exactly one category for the feature. [Source](https://dzlab.github.io/ml/2020/11/03/tfx-data-validation/)

In [ ]:
schema_gen.outputs['schema']._artifacts[0]

In [ ]:
schema_gen.outputs['schema']._artifacts[0].uri

In [ ]:
schema_uri = schema_gen.outputs['schema']._artifacts[0].uri

In [ ]:
# Get the schema pbtxt file from the SchemaGen output
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))

The domain for each feature within the schema is empty, will be good to update them.

In [ ]:
feature_names = ['age',
'sex',
'cp',
'trtbps',
'chol',
'fbs',
'restecg',
'thalachh',
'exng',
'oldpeak',
'slp',
'caa',
'thall',
'output']

categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exng', 'slp', 'caa', 'thall']

numerical_features = list(set(feature_names) - set(categorical_features))

numerical_features


## The features of the dataset and its respective description
We can assume that information on the features, such as whether the feature is categorical or numerical, or the domain of the feature is provided in the dataset metadata. In the future, we will explore how to attain this information by exploring the dataset using TFX. This is given under the `asset` folder as `heart_disease_features_descript.csv`.

| Feature  | Description                                                    | Type  | NumOrCat | Domain      | Domain_min | Domain_max |
|----------|----------------------------------------------------------------|-------|----------|-------------|------------|------------|
| age      | age in years                                                   | int   | Num      | na          | na         | na         |
| sex      | (1 = male; 0 = female)                                         | int   | Cat      | [1 0]       | 0          | 1          |
| cp       | chest pain type                                                | int   | Cat      | [0 1 2 3]   | 0          | 3          |
| trtbps   | resting blood pressure (in mm Hg on admission to the hospital) | int   | Num      | na          | na         | na         |
| chol     | serum cholestoral in mg/dl                                     | int   | Num      | na          | na         | na         |
| fbs      | (fasting blood sugar &gt; 120 mg/dl) (1 = true; 0 = false)     | int   | Cat      | [1 0]       | 0          | 1          |
| restecg  | resting electrocardiographic results                           | int   | Cat      | [0 1 2]     | 0          | 2          |
| thalachh | maximum heart rate achieved                                    | int   | Num      | na          | na         | na         |
| exng     | exercise induced angina (1 = yes; 0 = no)                      | int   | Cat      | [1 0]       | 0          | 1          |
| oldpeak  | ST depression induced by exercise relative to rest             | float | Num      | na          | na         | na         |
| slp      | the slope of the peak exercise ST segment                      | int   | Cat      | [0 1 2]     | 0          | 2          |
| caa      | number of major vessels (0-3) colored by flourosopy            | int   | Cat      | [0 1 2 3 4] | 0          | 4          |
| thall    | 3 = normal; 6 = fixed defect; 7 = reversable defect            | int   | Cat      | [0 1 2 3]   | 0          | 3          |
| output   | 1 (heart disease) or 0                                         | int   | Cat      | [1 0]       | 0          | 1          |

In [ ]:
features_metadata = pd.read_csv(os.path.join('.','assets','heart_disease_features_descript.csv'))

features_metadata.set_index(['Feature'], inplace=True)

features_metadata

In [ ]:
int(features_metadata.at['sex','Domain_max'])


Now, let's update the domains for the categorical features.

In [ ]:
for cat_feat in categorical_features:
    
    tfdv.set_domain(schema, cat_feat, schema_pb2.IntDomain(name=cat_feat,
                                                           min=0,
                                                           max=int(features_metadata.at[cat_feat,'Domain_max']),
                                                           is_categorical=True))

In [ ]:
_FEATURES_ONE_HOT_DEPTHS = {}

for cat_feat in categorical_features:
    
    _FEATURES_ONE_HOT_DEPTHS[cat_feat] = int(features_metadata.at[cat_feat,'Domain_max']) + 1
    
_FEATURES_ONE_HOT_DEPTHS

In [ ]:
tfdv.display_schema(schema)

Save the curated schema and overwite the old schema.

In [ ]:
tfdv.write_schema_text(schema, os.path.join(schema_uri, 'schema.pbtxt'))

In [ ]:
context.show(schema_gen.outputs['schema'])

<a id ='eg_val'></a>
## ExampleValidator

In [ ]:
# Instantiate ExampleValidator with the statistics and schema from the previous steps
example_validator = ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema = schema_gen.outputs['schema']
)
    
# Run the component
context.run(example_validator)

In [ ]:
# Visualize the output
context.show(example_validator.outputs['anomalies'])

## Transform

In [ ]:
# Some help so it's easier to copy and paste

for features_list in [categorical_features, numerical_features, feature_names]:
    print("[",end="")
    for feat_idx, each_feature in enumerate(features_list):
        if feat_idx == len(features_list)-1:
            print("'"+str(each_feature)+"'",end="")
        else:
            print("'"+str(features_list[feat_idx])+"'",end=",") 
    print("]")

In [ ]:
%%writefile {_constant_transform_module_file}
## This file is all about writing the constants neccessary to run
# the preprocessing_fn in the _features_transform_module_file.

# Features to bucketize
BUCKET_FEATURE_KEYS = ['age']

# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = {'age': 10}

# Feature to scale from 0 to 1
RANGE_FEATURE_KEYS = ['oldpeak','thalachh','chol','trtbps']

# Feature to apply one_hot encoding on
ONE_HOT_KEYS = ['sex','cp','fbs','restecg','exng','slp','caa','thall']

# ONE_HOT DEPTH
FEATURE_OH_DEPTHS = {'sex': 2,
'cp': 4,
'fbs': 2,
'restecg': 3,
'exng': 2,
'slp': 3,
'caa': 5,
'thall': 4}

# Feature to predict
LABEL_KEY = 'output'

def transformed_name(key):
    return key + '_xf'

In [ ]:
%%writefile {_features_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import constants_transform # The .py module file name NOT the variable to which that is assigned to.

_BUCKETIZED_FEATURE_KEYS = constants_transform.BUCKET_FEATURE_KEYS # 1
_BUCKETIZED_FEATURE_COUNT = constants_transform.FEATURE_BUCKET_COUNT
_SCALE_0_1_FEATURE_KEYS = constants_transform.RANGE_FEATURE_KEYS # 2
_SCALE_ONE_HOT_FEATURE_KEYS = constants_transform.ONE_HOT_KEYS # 3
_FEATURES_ONE_HOT_DEPTHS = constants_transform.FEATURE_OH_DEPTHS
_LABEL_KEY = constants_transform.LABEL_KEY
_transformed_name = constants_transform.transformed_name

def preprocessing_fn(inputs):

    features_dict = {}

    for feature in _SCALE_0_1_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using scaling of 0 to 1 function
        features_dict[_transformed_name(feature)] = tft.scale_to_0_1(
            data_col
        )

    for feature in _SCALE_ONE_HOT_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using one_hot
        features_dict[_transformed_name(feature)] = tf.one_hot(
            data_col,
            depth = _FEATURES_ONE_HOT_DEPTHS[feature]
        )

    for feature in _BUCKETIZED_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using vocabulary available in column
        # Hint: Use tft.compute_and_apply_vocabulary
        features_dict[_transformed_name(feature)] = tft.bucketize(
            data_col,
            _BUCKETIZED_FEATURE_COUNT[feature]
        )
        
    # No change in the label
    features_dict[_transformed_name(_LABEL_KEY)] = inputs[_LABEL_KEY]

    return features_dict

# def _fill_in_missing(x):
#     """Replace missing values in a SparseTensor and convert to a dense tensor.
#     Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
#     Args:
#         x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
#           in the second dimension.
#     Returns:
#         A rank 1 tensor where missing values of `x` have been filled in.
#     """
#     default_value = '' if x.dtype == tf.string else 0

#     return tf.squeeze(
#       tf.sparse.to_dense(
#           tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
#           default_value),
#       axis=1)

In [ ]:
# Instantiate the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=_features_transform_module_file
)

context.run(transform)

In [ ]:
context.show(transform)

In [ ]:
transform.outputs['transformed_examples'].get()[0]

In [ ]:
print(train_uri)
os.listdir(train_uri)

In [ ]:
transform_uri = transform.outputs['transformed_examples'].get()[0].uri

# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform_uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

In [ ]:
%%writefile {_constants_train_module_file}
## This file is all about writing the constants neccessary to run
# the preprocessing_fn in the _features_transform_module_file.

# Features to bucketize
BUCKET_FEATURE_KEYS = ['age']

# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = {'age': 10}

# Feature to scale from 0 to 1
RANGE_FEATURE_KEYS = ['oldpeak','thalachh','chol','trtbps']

# Feature to apply one_hot encoding on
ONE_HOT_KEYS = ['sex','cp','fbs','restecg','exng','slp','caa','thall']

# LIST CONTAINING ALL FEATURE NAMES
ALL_FEATURE_KEYS = ['age',
'sex',
'cp',
'trtbps',
'chol',
'fbs',
'restecg',
'thalachh',
'exng',
'oldpeak',
'slp',
'caa',
'thall']
#'output']

# ONE_HOT DEPTH
FEATURES_OH_DEPTHS = {'sex': 2,
'cp': 4,
'fbs': 2,
'restecg': 3,
'exng': 2,
'slp': 3,
'caa': 5,
'thall': 4}

# FEATURES INPUT SHAPES
FEATURES_INPUT_SHAPES = {'age': {'shape': 1},
'sex': {'shape': 2},
'cp': {'shape': 4},
'trtbps': {'shape': 1},
'chol': {'shape': 1},
'fbs': {'shape': 2},
'restecg': {'shape': 3},
'thalachh': {'shape': 1},
'exng': {'shape': 2},
'oldpeak': {'shape': 1},
'slp': {'shape': 3},
'caa': {'shape': 5},
'thall': {'shape': 4}}

# Feature to predict
LABEL_KEY = 'output'

# Model Specific Parameters
BATCH_SIZE = 32
HIDDEN_UNITS = [32, 32, 32]
LEARNING_RATE = 0.001
NUM_EPOCHS = 1000

def transformed_name(key):
    return key + '_xf'

def nll(y_true, y_pred):
    return -y_pred.log_prob(y_true)

In [ ]:
%%writefile {_model_trainer_module_file}
###############################################
# REFERENCE
# https://www.tensorflow.org/tfx/tutorials/tfx/components_keras
###############################################

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft
import tfx

from tfx_bsl.public import tfxio

# Imports related to the model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K
import tensorflow_probability as tfp

tfd = tfp.distributions
tfpl = tfp.layers

import constants_trainer

_BUCKETIZED_FEATURE_KEYS = constants_trainer.BUCKET_FEATURE_KEYS # 1
_BUCKETIZED_FEATURE_COUNT = constants_trainer.FEATURE_BUCKET_COUNT
_SCALE_0_1_FEATURE_KEYS = constants_trainer.RANGE_FEATURE_KEYS # 2
_SCALE_ONE_HOT_FEATURE_KEYS = constants_trainer.ONE_HOT_KEYS # 3
_FEATURES_ONE_HOT_DEPTHS = constants_trainer.FEATURES_OH_DEPTHS
_FEATURES_INPUT_SHAPES = constants_trainer.FEATURES_INPUT_SHAPES
_LABEL_KEY = constants_trainer.LABEL_KEY
_transformed_name = constants_trainer.transformed_name
_ALL_FEATURE_KEYS = constants_trainer.ALL_FEATURE_KEYS

# Model Specific Parameters
_BATCH_SIZE = constants_trainer.BATCH_SIZE
_HIDDEN_UNITS = constants_trainer.HIDDEN_UNITS
_LEARNING_RATE = constants_trainer.LEARNING_RATE
_NUM_EPOCHS = constants_trainer.NUM_EPOCHS
_nll = constants_trainer.nll

def _transformed_names(keys):
    return [_transformed_name(key) for key in keys]

def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""

    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(_LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)

    return serve_tf_examples_fn

def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.trainer.fn_args_utils.DataAccessor,
              # Updated tfx.components.DataAccessor to tfx.components.trainer.fn_args_utils.DataAccessor
              tf_transform_output: tft.TFTransformOutput,
              batch_size: _BATCH_SIZE) -> tf.data.Dataset:
    
    """Generates features and label for tuning/training.

    Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
    dataset to combine in a single batch

    Returns:
    A dataset that contains (features, indices) tuple where features is a
    dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY),
          drop_final_batch=False),
      tf_transform_output.transformed_metadata.schema)

def _create_model_inputs(feature_names = _ALL_FEATURE_KEYS): 
    inputs = {}
    for feature_name in feature_names:
        # Model takes 'age_fx' as names for inputs instead of 'age', therefore
        # _transformed_name() is there to append 'fx' after the feature name.
        inputs[_transformed_name(feature_name)] = layers.Input(
            name=_transformed_name(feature_name), shape=(1,_FEATURES_INPUT_SHAPES[feature_name]['shape']),
            dtype=tf.float32
        )
    return inputs

def _create_bnn_model(train_size = _BATCH_SIZE, hidden_units = _HIDDEN_UNITS,
                     learning_rate = _LEARNING_RATE, loss = _nll):
    
    divergence_fn = lambda q, p, _:tfd.kl_divergence(q, p) / train_size
    
    inputs = _create_model_inputs()
    features_ = keras.layers.concatenate(list(inputs.values()))
    features_ = keras.layers.BatchNormalization()(features_)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features_ = tfpl.DenseReparameterization(
            units = units, activation = 'relu',
            kernel_prior_fn = tfpl.default_multivariate_normal_fn,
            kernel_posterior_fn = tfpl.default_mean_field_normal_fn(is_singular = False),
            kernel_divergence_fn = divergence_fn,
            bias_prior_fn = tfpl.default_multivariate_normal_fn,
            bias_posterior_fn = tfpl.default_mean_field_normal_fn(is_singular = False),
            bias_divergence_fn = divergence_fn
        )(features_)
    
    distribution_params = tfpl.DenseReparameterization(
            units = tfp.layers.OneHotCategorical.params_size(2), activation = None,
            kernel_prior_fn = tfpl.default_multivariate_normal_fn,
            kernel_posterior_fn = tfpl.default_mean_field_normal_fn(is_singular = False),
            kernel_divergence_fn = divergence_fn,
            bias_prior_fn = tfpl.default_multivariate_normal_fn,
            bias_posterior_fn = tfpl.default_mean_field_normal_fn(is_singular = False),
            bias_divergence_fn = divergence_fn
        )(features_)

    outputs = tfp.layers.OneHotCategorical(2,
                                          convert_to_tensor_fn=tfd.Distribution.mode)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics = ['accuracy'],
        experimental_run_tf_function=False
    )
    
    return model

# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.trainer.fn_args_utils.FnArgs):
           # Updated tfx.components.FnArgs to tfx.components.trainer.fn_args_utils.FnArgs
    """Train the model based on given args.

    Args:
    fn_args: Holds args used to train the model as name/value pairs.
    """

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, _BATCH_SIZE)
    eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, _BATCH_SIZE)


    model = _create_bnn_model()
    
    model.fit(
    train_dataset,
    steps_per_epoch=fn_args.train_steps,
    validation_data=eval_dataset,
    validation_steps=fn_args.eval_steps,
    epochs = 10) #,
    
    signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
              tf.TensorSpec(shape=[None],
                            dtype=tf.string,
                            name='examples')
          )
    }
    
    tf.keras.models.save_model(model, fn_args.serving_model_dir, save_format='tf', signatures=signatures)

In [ ]:
train_steps = int(np.ceil(212//32))
eval_steps = int(np.ceil(91//32))

trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_model_trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=train_steps),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=eval_steps)
)

context.run(trainer)

In [ ]:
trainer.outputs['model'].get()[0].uri

<a name = 'pusher'></a>

# Pusher

In [ ]:
pusher = Pusher(
    model=trainer.outputs['model'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

In [ ]:
push_uri = pusher.outputs.pushed_model.get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
    pp.pprint(item)

In [ ]:
# Get prediction function from serving
f = model.signatures['serving_default']

# Complete Pipeline



## Running it locally

In [ ]:
from tfx.orchestration.local.local_dag_runner import LocalDagRunner

In [ ]:
# https://github.com/tensorflow/tfx/blob/master/tfx/examples/chicago_taxi_pipeline/

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, transform_module_file: str, trainer_module_file: str,
                     serving_model_dir: str, metadata_path: str):
    """Implements the heart disease pipeline with TFX."""
    
    # Brings data into the pipeline or otherwise joins/converts training data.
    example_gen = tfx.components.CsvExampleGen(
        input_base=data_root
    )

    # Computes statistics over data for visualization and example validation.
    statistics_gen = tfx.components.StatisticsGen(
        examples=example_gen.outputs['examples']
    )

  # Import the schema.
    schema_importer = tfx.dsl.components.common.importer.Importer(
        source_uri=schema_path,
        artifact_type=tfx.types.standard_artifacts.Schema
    ).with_id('schema_importer')

  # Performs anomaly detection based on statistics and data schema.
    example_validator = tfx.components.ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_importer.outputs['result']
    )

  # NEW: Transforms input data using preprocessing_fn in the 'module_file'.
    transform = tfx.components.Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_importer.outputs['result'],
        materialize=False,
        module_file=transform_module_file
    )
    
    train_steps = int(np.ceil(212//32))
    eval_steps = int(np.ceil(91//32))

    # Uses user-provided Python function that trains a model.
    trainer = tfx.components.Trainer(
        module_file=trainer_module_file,
        examples=example_gen.outputs['examples'],

    # NEW: Pass transform_graph to the trainer.
        transform_graph=transform.outputs['transform_graph'],

        train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=train_steps),
        eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=eval_steps)
    )

    # Pushes the model to a filesystem destination.
    pusher = tfx.components.Pusher(
        model=trainer.outputs['model'],
        push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=serving_model_dir)
        )
    )

    components = [
        example_gen,
        statistics_gen,
        schema_importer,
        example_validator,
        transform,  # NEW: Transform component was added to the pipeline.
        trainer,
        pusher
    ]
    
    # https://www.tensorflow.org/tfx/guide/build_tfx_pipeline
    # https://github.com/tensorflow/tfx/blob/master/tfx/orchestration/pipeline.py
    
    return tfx.orchestration.pipeline.Pipeline( # Changed tfx.dsl.Pipeline to tfx.orchestration.pipeline.Pipeline
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(metadata_path),
        components=components
    )

In [ ]:
_PIPELINE_NAME = 'local_HD_BNN_pipeline'
_PIPELINE_ROOT = os.path.join(os.getcwd(),'local_HD_BNN_pipeline')
_DATA_ROOT = os.path.join(_PIPELINE_ROOT, 'data')
_SCHEMA_PATH = os.path.join(_PIPELINE_ROOT, 'schema')
_SERVING_MODEL_DIR = os.path.join(_PIPELINE_ROOT, 'serving_model')
_METADATA_PATH = os.path.join(_PIPELINE_ROOT, 'metadata.sqlite')

_transform_module_file = 'features_transform.py'
_trainer_module_file = 'model_trainer.py'

In [ ]:
LocalDagRunner().run(
    _create_pipeline(
        pipeline_name=_PIPELINE_NAME,
        pipeline_root=_PIPELINE_ROOT,
        data_root=_DATA_ROOT,
        schema_path=_SCHEMA_PATH,
        transform_module_file=_transform_module_file,
        trainer_module_file=_trainer_module_file,
        serving_model_dir=_SERVING_MODEL_DIR,
        metadata_path=_METADATA_PATH
    )
)

An error is raised in the running of the above cell, I had raised an issue with regards to this error to TFX: [TFX-3999](https://github.com/tensorflow/tfx/issues/3999)

## References

This notebook took reference from, copied and adapted, some of the codes from the following:

1. Machine Learning Data Lifecycle in Production by DeepLearning.AI, assignments from Week 2 and Week 3 ([Link](https://www.coursera.org/learn/machine-learning-data-lifecycle-in-production/home/welcome))
2. Collab Notebook ([Link](https://colab.research.google.com/gist/rafiqhasan/2164304ede002f4a8bfe56e5434e1a34/dl-e2e-taxi-dataset-tfx-e2e.ipynb)) | [Youtube Tutorial](https://www.youtube.com/watch?v=VrBoQCchJQU) by Hasan
3. Tensorflow Tutorial on TFX Pipeline for Chicago Taxi Dataset ([Link](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras))
4. Tensorflow Tutorial on TFX Pipeline for Penguin ([Link](https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_tft))